In [2]:
! conda install -y gdown

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py37hc8dfbb8_0         151 KB  conda-forge
    gdown-3.11.1               |     pyh9f0ad1d_0          12 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         308 KB

The following NEW packages will be INSTALLED:

  gdown              conda-forge/noarch::gdown-3.11.1-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5.2-hecda079_0 --> 2020.6.20-hecda079_0
  certifi                         2020.4.5.2-py37hc8dfbb8_0 --> 2020.6.20-py37hc8dfbb8_0



certifi-2020.6.20    | 151 KB    | #

In [7]:
import gdown
#url = 'https://drive.google.com/file/d/1M1nfh0XEq4dNLowuCeoRdQbpt0sPxZj7'
url = 'https://drive.google.com/uc?id=1M1nfh0XEq4dNLowuCeoRdQbpt0sPxZj7'
output = 'test.csv'

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1M1nfh0XEq4dNLowuCeoRdQbpt0sPxZj7
To: /kaggle/working/test.csv
100%|██████████| 15.8k/15.8k [00:00<00:00, 14.4MB/s]


'test.csv'

In [8]:
url = 'https://drive.google.com/uc?id=1VKZm08RjH5E_tWB9XDfmj47ydd0YYYai'
output = 'train.csv'

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1VKZm08RjH5E_tWB9XDfmj47ydd0YYYai
To: /kaggle/working/train.csv
100%|██████████| 36.7k/36.7k [00:00<00:00, 23.9MB/s]


'train.csv'

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
!pip install transformers==2.11.0 --quiet
!pip install simpletransformers==0.41.0 --quiet
!pip install nlpaug numpy matplotlib python-dotenv --quiet
!pip install pyspellchecker --quiet


In [11]:
import random
import torch
from simpletransformers.classification import ClassificationModel
import pandas as pd
import numpy as np
#import logging
from tqdm import tqdm_notebook
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [12]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.special import softmax
import sklearn
from sklearn.metrics import log_loss
import numpy as np

In [13]:
test=pd.read_csv('/kaggle/working/test.csv')
train=pd.read_csv('//kaggle/working/train.csv')

In [14]:
train.head()

,ID,text,label
0,SUAVK39Z,I feel that it was better I dieAm happy,Depression
1,9JDAGUV3,Why do I get hallucinations?,Drugs
2,419WR1LQ,I am stresseed due to lack of financial suppor...,Depression
3,6UY7DX6Q,Why is life important?,Suicide
4,FYC0FTFB,How could I be helped to go through the depres...,Depression


In [15]:
le = preprocessing.LabelEncoder()
le.fit(train['label'])
train['label']=le.transform(train['label'])
print(list(le.classes_))

['Alcohol', 'Depression', 'Drugs', 'Suicide']


In [16]:

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_all(79)

In [68]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [17]:
train.duplicated().sum()

0

In [18]:
train=train.drop(['ID'],axis=1)
train.head()

,text,label
0,I feel that it was better I dieAm happy,1
1,Why do I get hallucinations?,2
2,I am stresseed due to lack of financial suppor...,1
3,Why is life important?,3
4,How could I be helped to go through the depres...,1


In [19]:
test.head()

,ID,text
0,02V56KMO,How to overcome bad feelings and emotions
1,03BMGTOK,I feel like giving up in life
2,03LZVFM6,I was so depressed feel like got no strength t...
3,0EPULUM5,I feel so low especially since I had no one to...
4,0GM4C5GD,can i be successful when I am a drug addict?


In [20]:
final=pd.DataFrame()
final['ID']=test['ID']
test1=test.drop(['ID'],axis=1)
test1['label']=0

In [21]:
test1.head()

,text,label
0,How to overcome bad feelings and emotions,0
1,I feel like giving up in life,0
2,I was so depressed feel like got no strength t...,0
3,I feel so low especially since I had no one to...,0
4,can i be successful when I am a drug addict?,0


In [22]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
        
text = "corect me plese"
correct_spellings(text)

'correct me please'

In [23]:
train_fr=train.copy()

In [24]:
#train_fr['text']=train_fr['text'].apply(lambda x : correct_spellings(x))
#train_fr.head()

In [25]:
train_fr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616 entries, 0 to 615
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    616 non-null    object
 1   label   616 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.8+ KB


In [28]:
from tqdm import tqdm_notebook
aug_xlnet = nas.ContextualWordEmbsForSentenceAug(model_path='xlnet-base-cased')
#aug = naw.ContextualWordEmbsAug(model_path='roberta-base', action="substitute")
#augmented_text = aug.augment(text)
#augmented_text = aug.augment(text)
texts = train_fr['text'].values
aug_text_xlnet=[]
#for ii in tqdm_notebook(range(-8,-1)):
 #   print(texts[ii])
  #  print(aug.augment(texts[ii],n=1),'\n')

for ii in tqdm_notebook(range(616)):
  aug_text_xlnet.append(aug_xlnet.augment(texts[ii],n=1))

In [29]:
aug_text_xlnet

['I feel that it was better I dieAm happy with my life What do you think on your life , love , beauty and fortune?',
 'Why do I get hallucinations? And why does Scott have intense tremor s eptic - panic attack , shock disorder , flashback s are discussed and pointed out by the entire movie at this particular',
 'I am stresseed due to lack of financial support in school this math or science class , why do I skip the gym instead of school this science class for six days it seems like about a year since this lesson',
 'Why is life important? Why is this important to us?',
 'How could I be helped to go through the depression? This is really a job that I love so much going through the mental illness.',
 'What are the effects of depression to ones health including the weight or height of their joint s cap and socket.',
 'Why is everything so hard to deal with in this life?',
 'I feel emotionally overwhelmed by the rest of the website.',
 'How to manage taking alcohol? Well , until you buy yo

In [30]:
from tqdm import tqdm_notebook
#aug_xlnet = nas.ContextualWordEmbsForSentenceAug(model_path='xlnet-base-cased')
aug_gpt = nas.ContextualWordEmbsForSentenceAug(model_path='gpt2')
#augmented_text = aug.augment(text)
#augmented_text = aug.augment(text)
texts = train_fr['text'].values
aug_text_gpt=[]
#for ii in tqdm_notebook(range(-8,-1)):
 #   print(texts[ii])
  #  print(aug.augment(texts[ii],n=1),'\n')

for ii in tqdm_notebook(range(616)):
  aug_text_gpt.append(aug_gpt.augment(texts[ii],n=1))

In [31]:
aug_text_gpt

['I feel that it was better I dieAm happy.',
 'Why do I get hallucinations? I got hallucinations.',
 'I am stresseed due to lack of financial support in school due to need to support life in care ful ness " or I was n \'t sure what to say " ". He told me of where to start "',
 'Why is life important? Yes ." ( n ep hi ) " This is a simple reality.',
 'How could I be helped to go through the depression? How could I be helped to go through the dep ression ?" I think I " all I " know I " st have been here I is now here',
 'What are the effects of depression to ones health and / or mental health?',
 'Why is everything so hard to deal with in this life today ?" she asked the man off.',
 'I feel emotionally overwhelmed ," he says.',
 'How to manage taking alcohol? Check out Find out where to att end.',
 'Is heaven open for us who smoke bhang? The reason is right.',
 'How to survive without money? There are an old may just come soon on the big most day every day and more information on for all

In [33]:
#create new df 
df_aug_xlnet = pd.DataFrame({'text':aug_text_xlnet})
df_aug_xlnet['label']=train_fr['label']
df_aug_xlnet.head()

,text,label
0,I feel that it was better I dieAm happy with m...,1
1,Why do I get hallucinations? And why does Scot...,2
2,I am stresseed due to lack of financial suppor...,1
3,Why is life important? Why is this important t...,3
4,How could I be helped to go through the depres...,1


In [34]:
df_aug_xlnet.to_csv('df_aug_xlnet.csv',index=False)

In [35]:
#create new df 
df_aug_gpt = pd.DataFrame({'text':aug_text_gpt})
df_aug_gpt['label']=train_fr['label']
df_aug_gpt.head()

,text,label
0,I feel that it was better I dieAm happy.,1
1,Why do I get hallucinations? I got hallucinati...,2
2,I am stresseed due to lack of financial suppor...,1
3,"Why is life important? Yes ."" ( n ep hi ) "" Th...",3
4,How could I be helped to go through the depres...,1


In [36]:
df_aug_gpt.to_csv('df_aug_gpt.csv',index=False)

In [37]:
train_aug = pd.concat([train, df_aug_gpt, df_aug_xlnet], ignore_index=True)
train_aug.head()

,text,label
0,I feel that it was better I dieAm happy,1
1,Why do I get hallucinations?,2
2,I am stresseed due to lack of financial suppor...,1
3,Why is life important?,3
4,How could I be helped to go through the depres...,1


In [38]:
train_aug.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1848 entries, 0 to 1847
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1848 non-null   object
 1   label   1848 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 29.0+ KB


In [39]:
train_aug = train_aug.reindex(np.random.permutation(train_aug.index))
train_aug= train_aug.reset_index(drop=True)
train_aug.head()

,text,label
0,If the person i wronged will forgive me and ac...,3
1,What are the effects of depression?,1
2,I am overwhelmed by life. I am overwhelmed by ...,3
3,I had two miscarriages,1
4,Effects of alcohol on my body health,0


In [40]:
train_aug.to_csv('train_aug.csv',index=False)

In [69]:
# Optional model configuration
model_args = {
    "num_train_epochs": 3, 
    'fp16': False, 
    "save_eval_checkpoints": False,
     "save_model_every_epoch": False,
      'reprocess_input_data': True,
       'overwrite_output_dir': True,
       'manual_seed': 79,
       'max_seq_length': 20,
       "silent": True, 
    "learning_rate":2e-5,
}

In [74]:
%%time
err=[]
y_pred_tot=[]
torch.cuda.empty_cache()
kf = StratifiedKFold(n_splits=15, shuffle=True, random_state=79)
for train_index, test_index in kf.split(train_aug, train_aug['label']):
  #print("TRAIN:", train_index, "TEST:", test_index)
  #X_train, X_test = train_data[train_index], train_data[test_index]
  #y_train, y_test = train_data[train_index], train_data[test_index]
  train1_trn, train1_val = train_aug.iloc[train_index], train_aug.iloc[test_index]
  #print(train1_trn)
  model_rb = ClassificationModel('roberta', 'roberta-base', use_cuda=True,num_labels=4, weight=[2.514, 1, 6.067, 5.333],args=model_args)
  model_rb.train_model(train1_trn)
  result, raw_outputs_val, _ = model_rb.eval_model(train1_val)
  #print(raw_outputs_val)
  raw_outputs_val = softmax(raw_outputs_val,axis=1)
  #print(raw_outputs_val)
  #print(train1_val['labels'])
  print(f"Log_Loss: {log_loss(train1_val['label'], raw_outputs_val)}")
  print(result)
  err.append(log_loss(train1_val['label'], raw_outputs_val))
  raw_outputs_test = model_rb.eval_model(test1)[1]
  raw_outputs_test = softmax(raw_outputs_test,axis=1)
  y_pred_tot.append(raw_outputs_test)
print("Mean LogLoss: ",np.mean(err))

Log_Loss: 0.09274848900789831
{'mcc': 0.9604094984395173, 'eval_loss': 0.23833402712625684}
Log_Loss: 0.041252281267530166
{'mcc': 0.9734816082121471, 'eval_loss': 0.07405792878125794}
Log_Loss: 0.04148453541453027
{'mcc': 0.973440473364161, 'eval_loss': 0.11104246480681468}
Log_Loss: 0.13790299528282196
{'mcc': 0.9454338129772303, 'eval_loss': 0.19639055318475584}
Log_Loss: 0.028548296598236766
{'mcc': 0.9864910280724238, 'eval_loss': 0.03739403744839365}
Log_Loss: 0.09507745744071795
{'mcc': 0.9589858963040647, 'eval_loss': 0.1499416695223772}
Log_Loss: 0.10394038172340309
{'mcc': 0.9591035389671443, 'eval_loss': 0.21441218953987118}
Log_Loss: 0.33996995053219997
{'mcc': 0.8770851830276597, 'eval_loss': 0.6245549049453984}
Log_Loss: 0.10079147673412643
{'mcc': 0.9727338785257428, 'eval_loss': 0.2054400929628173}
Log_Loss: 0.1905290323178988
{'mcc': 0.9480975193195851, 'eval_loss': 0.20365526646492071}
Log_Loss: 0.10086318228168534
{'mcc': 0.9459306335416636, 'eval_loss': 0.2250541155

In [75]:
to_submit =np.mean(y_pred_tot,0)

In [76]:
final['Depression']=to_submit[:,1]
final['Alcohol']=to_submit[:,0]
final['Suicide']=to_submit[:,3]
final['Drugs']=to_submit[:,2]
final.head(10)

,ID,Depression,Alcohol,Suicide,Drugs
0,02V56KMO,0.699002,0.001428,0.299091,0.000479
1,03BMGTOK,0.999249,0.000173,0.000295,0.000284
2,03LZVFM6,0.999243,0.000168,0.000293,0.000296
3,0EPULUM5,0.999256,0.000177,0.000296,0.000270
4,0GM4C5GD,0.000483,0.067214,0.000397,0.931906
5,0IYM8BYT,0.999163,0.000173,0.000422,0.000241
6,0JFJVD28,0.998999,0.000153,0.000539,0.000309
7,0OZGUFOF,0.999225,0.000186,0.000287,0.000303
8,0WP999ZX,0.000171,0.999142,0.000384,0.000303
9,0ZA45Y5F,0.000151,0.999067,0.000303,0.000479


In [77]:
final.to_csv('aug20seq-roberta_base_2e5_3ep_15f_weighted.csv',index=False)